# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\инциденты\Экс4.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,2.000000,6.0,7.000,7.0,5.0,9
1,Экспертиза промышленной безопасности,0.500000,1.000000,7.0,3.000,6.0,9.0,7
2,Готовность к ликвидации последствий аварий и и...,0.166667,0.142857,1.0,1.000,5.0,7.0,8
3,Оформление информационными знаками и знаками б...,0.142857,0.333333,1.0,1.000,8.0,4.0,5
4,Содержание объектов и оборудования,0.142857,0.166667,0.2,0.125,1.0,1.0,7


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              2.000000   
1       0.500000                              1.000000   
2       0.166667                              0.142857   
3       0.142857                              0.333333   
4       0.142857                              0.166667   
5       0.200000                              0.111111   
6       0.111111                              0.142857   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           6.000000         
1                                           7.000000         
2                                           1.000000         
3                                           1.000000         
4                                           0.200000         
5                                           0.142857         
6                                           0.125000         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,2.000000,6.000000,7.000,7.000000,5.000000
1,0.500000,1.000000,7.000000,3.000,6.000000,9.000000
2,0.166667,0.142857,1.000000,1.000,5.000000,7.000000
3,0.142857,0.333333,1.000000,1.000,8.000000,4.000000
4,0.142857,0.166667,0.200000,0.125,1.000000,1.000000
5,0.200000,0.111111,0.142857,0.250,1.000000,1.000000
6,0.111111,0.142857,0.125000,0.200,0.142857,0.111111


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  2.263492
Экспертиза промышленной безопасности                          3.896825
Готовность к ликвидации последствий аварий и инцидентов      15.467857
Оформление информационными знаками и знаками безопасности    12.575000
Содержание объектов и оборудования                           28.142857
Охранные зоны (ОЗ)                                           27.111111
Организация рабочего процесса и безопасности персонала       46.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,2.000000,6.0,7.000,7.0,5.0,9,0.441795,0.513238,0.387901,0.556660,0.248731,0.184426,0.195652
1,0.500000,1.000000,7.0,3.000,6.0,9.0,7,0.220898,0.256619,0.452551,0.238569,0.213198,0.331967,0.152174
2,0.166667,0.142857,1.0,1.000,5.0,7.0,8,0.073633,0.036660,0.064650,0.079523,0.177665,0.258197,0.173913
3,0.142857,0.333333,1.0,1.000,8.0,4.0,5,0.063114,0.085540,0.064650,0.079523,0.284264,0.147541,0.108696
4,0.142857,0.166667,0.2,0.125,1.0,1.0,7,0.063114,0.042770,0.012930,0.009940,0.035533,0.036885,0.152174


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.441795,0.513238,0.387901,0.556660,0.248731,0.184426,0.195652
1,0.220898,0.256619,0.452551,0.238569,0.213198,0.331967,0.152174
2,0.073633,0.036660,0.064650,0.079523,0.177665,0.258197,0.173913
3,0.063114,0.085540,0.064650,0.079523,0.284264,0.147541,0.108696
4,0.063114,0.042770,0.012930,0.009940,0.035533,0.036885,0.152174
5,0.088359,0.028513,0.009236,0.019881,0.035533,0.036885,0.195652
6,0.049088,0.036660,0.008081,0.015905,0.005076,0.004098,0.021739


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.441795,0.513238,0.387901,0.556660,0.248731,0.184426,0.195652,0.361201
1,0.220898,0.256619,0.452551,0.238569,0.213198,0.331967,0.152174,0.266568
2,0.073633,0.036660,0.064650,0.079523,0.177665,0.258197,0.173913,0.123463
3,0.063114,0.085540,0.064650,0.079523,0.284264,0.147541,0.108696,0.119047
4,0.063114,0.042770,0.012930,0.009940,0.035533,0.036885,0.152174,0.050478


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

3.298376167420815
2.6444261149603507
1.1679802486710682
1.1238575753320712
0.38587852170360193
0.43972010143890206
0.15133278009766146


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

9.21157150962447


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.3685952516040783
1.4142857142857144
0.2606229051746008


In [20]:
# Получилось, что значение CR=0.26, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами